# 弹幕游戏的操作序列分析

## 问题背景

游戏正在耗费人类越来越多的时间，所以对于人类在电脑游戏这一形式化的系统上的行为的研究，无论对于人类自身还是游戏的相关产业都是否有用。
而在对于游戏可能产生的数据，除了在现实中的响应外，最重要的应该是游戏内部的操作序列。很多游戏可以记录玩家的操作序列，
通常被称为“回放”（replay）。这些回放仅仅包含操作信息，可以重新被游戏读取模拟玩家的操作，以无损失的重现一场游戏的过程。
这与现在流行的通过共享对游戏录制的视频是完全不同的，视频不用具有游戏即可以播放，但仅能记录画面的信息，
而这些画面信息完全可以由大小小多了的操作序列通过运行游戏重造出来。早期网络的网速较慢，出于与现在视频流行类似的共享需求，
回放技术得到了发展。